# Parte I
<b>Considerando come trading date il 20-Dec-2022, si prendano due CDS standardizzati (rolling su date unadjusted ‘credit IMM’): il primo con maturity 3Y (20/Dec/2025), il secondo con maturity 5Y (20/Dec/2027). Entrambi i CDS abbiano una cedola contrattuale annualizzata pari all’1% del nozionale e si ipotizzi per il contratto una recovery convenzionale pari al 40%. Si ipotizzi inoltre che la curva dei tassi di interesse sia deterministica con tassi zero-coupon pari al 3% annuo. Sapendo che il premio Upfront quotato dal mercato vale 6.80% del nozionale per il primo CDS e 9.60% del nozionale per il secondo</b>

In [79]:
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np
import pandas as pd
from scipy.integrate import quad
import matplotlib as plt 
import math as mt


## Esercizio I.I

<b>Si determinino mediante ‘bootstrap’ due livelli di default intensity λ1 e λ2, validi sugli intervalli [0y,3y] e [3y,5y] rispettivamente, compatibili con tali quotazioni (per tale scopo si potrà assumere una regola ‘end-of-period’ per i pagamenti della default leg)</b>

$$
PV_{\text{Default Leg}} = LGD \times \sum_{i=1}^{n3Y} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right]
$$

$$
PV_{\text{Coupon Leg}} = s \times \sum_{i=1}^{n3Y} \alpha_i \cdot P(t, t_i) \cdot e^{-\lambda_{3Y} \cdot (t_i - t)}
$$


$$
UF^{3Y} = PV_{\text{Default Leg}} - PV_{\text{Coupon Leg}}
$$

$$
% Definizione del valore attuale del premio upfront per il CDS a 5 anni
PV_{\text{upfront, 5y}}(\lambda_{5Y}) = LGD \left( \sum_{i=1}^{3} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right] + \sum_{i=4}^{5} P(t, t_i) \left[ e^{-\lambda_{5Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{5Y} \cdot (t_i - t)} \right] \right) - s \left( \sum_{i=1}^{3} \alpha_i P(t, t_i) e^{-\lambda_{3Y} \cdot (t_i - t)} + \sum_{i=4}^{5} \alpha_i P(t, t_i) e^{-\lambda_{5Y} \cdot (t_i - t)} \right)
$$

$$
UF^{5Y} = PV_{\text{upfront, 5y}}(\lambda_{5Y})
$$

In [80]:
r = 0.03  # Tasso di interesse zero-coupon annuale
recovery_rate = 0.4  # Tasso di recupero
LGD = 1 - recovery_rate  # Loss Given Default (1 - tasso di recupero)
spread = 0.01  # Spread del CDS, cedola annuale
upfront_3y = 0.068  # Premio upfront osservato per il CDS a 3 anni
upfront_5y = 0.096  # Premio upfront osservato per il CDS a 5 anni
n3Y = 3  # Numero di anni fino alla scadenza del CDS a 3 anni
n5Y = 5  # Numero di anni fino alla scadenza del CDS a 5 anni

Definiamo una funzione per calcolare il prezzo di un zero-coupon bond. Questo è usato per scontare i flussi di cassa futuri al valore presente.

In [81]:
# Funzione che calcola il prezzo di uno zero-coupon bond che matura al tempo t
def zero_coupon_bond_price(r, t):
    return np.exp(-r * t)

In [82]:
def pv_upfront_combined(lambda_rates, r, LGD, spread_annual, upfront_3y, upfront_5y, n3Y, n5Y):
    lambda_3Y, lambda_5Y = lambda_rates
    year_fraction = np.array([i / 360 for i in range(1, n5Y + 1)])
    spread = spread_annual * year_fraction
    
    # Flussi di cassa per i primi 3 anni con lambda_3Y
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_coupon_3y = sum([spread[i-1] * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    pv_upfront_3y = LGD * pv_loss_3y - pv_coupon_3y

    # Flussi di cassa per gli anni 3-5 con lambda_5Y
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_coupon_5y = sum([spread[i-1] * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    pv_upfront_5y = LGD * pv_loss_5y - pv_coupon_5y

    return np.array([upfront_3y - pv_upfront_3y, upfront_5y - (pv_upfront_3y + pv_upfront_5y)])


In [83]:
lambda_initial_guesses = [0.1, 0.1]
lambda_solutions = fsolve(pv_upfront_combined, lambda_initial_guesses, args=(r, LGD, spread, upfront_3y, upfront_5y, 3, 5))

In [84]:
lambda_3Y, lambda_5Y = lambda_solutions


data = [[lambda_3Y, lambda_5Y]] 
df = pd.DataFrame(data, columns=['λ1', 'λ2'])
df = df.style.set_caption('Default Intensity')
df

,λ1,λ2
0,0.042782,0.030342


Per verificare l'accuratezza dei valori di $\lambda$ calcolati per i CDS a 3 anni e a 5 anni, abbiamo implementato una funzione aggiuntiva non esplicitamente richiesta dal problema originale. Questa funzione, denominata `calculate_upfront`, utilizza $\lambda$ ottenuti per calcolare il valore dell'upfront dei CDS, che poi viene confrontato con i valori di upfront forniti nel problema.

L'obiettivo di questa verifica è assicurarsi che i $\lambda$ calcolati siano corretti e che i metodi utilizzati per la loro determinazione siano validi. I valori di upfront calcolati per i CDS a 3 anni e a 5 anni sono risultati rispettivamente di circa $0.068$ e $0.096$, molto vicini ai valori forniti dal problema ($0.068$) per il CDS a 3 anni e $0.096$ per il CDS a 5 anni.

Questa corrispondenza tra i valori calcolati e quelli forniti conferma che i  $\lambda$ determinati sono accurati. La coerenza tra i valori di upfront calcolati e quelli osservati indica che l'approccio adottato per il calcolo dei $\lambda$ è corretto e che i valori ottenuti sono affidabili.

In [85]:
def calculate_upfront(lambda_3Y, lambda_5Y, r, LGD, spread_annual, n3Y, n5Y):
    year_fraction = np.array([i / 360 for i in range(1, n5Y + 1)])
    spread = spread_annual * year_fraction
    
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_coupon_3y = sum([spread[i-1] * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_coupon_5y = sum([spread[i-1] * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    
    upfront_calculated = LGD * (pv_loss_3y + pv_loss_5y) - (pv_coupon_3y + pv_coupon_5y)

    return upfront_calculated

In [86]:
# Calcoliamo l'upfront
upfront_calculated_3y = calculate_upfront(lambda_3Y, lambda_5Y, r, LGD, spread, 3, 3)  # per 3 anni usando solo lambda_3Y
upfront_calculated_5y = calculate_upfront(lambda_3Y, lambda_5Y, r, LGD, spread, 3, 5)  # per 5 anni usando entrambi i lambda

In [87]:
data = [['Upfront 3 anni', upfront_calculated_3y, upfront_3y, '%.15f' % (upfront_calculated_3y - upfront_3y)],
        ['Upfront 5 anni', upfront_calculated_5y, upfront_5y, '%.15f' % (upfront_calculated_5y - upfront_5y)]] 
df = pd.DataFrame(data, columns=[' ', 'Upfront calcolato', 'Upfront fornito', 'Differenza'])
df = df.style.set_caption('Calcolo Upfront')
df

,,Upfront calcolato,Upfront fornito,Differenza
0,Upfront 3 anni,0.068000,0.068000,-0.000000000000013
1,Upfront 5 anni,0.096000,0.096000,0.000000000000139



## Esercizio I.II 
<b>Si calcolino le probabilità di sopravvivenza a 1y, 2y, 3y, 4y, 5y.</b>

$$
% Calcolo della probabilità di sopravvivenza fino a un certo tempo T
\text{Surv}(T) = 
\begin{cases} 
e^{-\lambda_{3Y} \cdot T}, & \text{se } T \leq 3 \\
e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (T - 3)}, & \text{se } T > 3 
\end{cases}

$$

Questa formula fornisce la rappresentazione matematica di come calcoliamo la probabilità di sopravvivenza per il CDS, sia per i primi tre anni utilizzando 
$\lambda_{3Y}$ sia per gli anni successivi fino a cinque anni utilizzando $\lambda_{5Y}$

$$
\text{Probabilità di sopravvivenza a 1 anno} = e^{-\lambda_{3Y} \cdot 1}\\
\text{Probabilità di sopravvivenza a 2 anni} = e^{-\lambda_{3Y} \cdot 2}\\
\text{Probabilità di sopravvivenza a 3 anni} = e^{-\lambda_{3Y} \cdot 3}\\
\text{Probabilità di sopravvivenza a 4 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (4 - 3)}\\
\text{Probabilità di sopravvivenza a 5 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (5 - 3)}
$$



Utilizziamo la proprietà dei logaritmi e degli esponenziali per combinare le probabilità di sopravvivenza in intervalli di tempo consecutivi, assumendo che l'intensità di default sia costante in ciascun intervallo. Questo ci permette di calcolare la probabilità complessiva che la controparte sopravviva senza default fino al tempo $T$, dato che ha sopravvissuto fino a 3 anni, e poi calcolare la probabilità che continui a sopravvivere fino a $T$ utilizzando $\lambda_{5Y}$

In [88]:
# Funzione per calcolare la probabilità di sopravvivenza fino a un certo tempo T
def calculate_survival_probability(lambda_1, lambda_2, T, n3Y):
    # Se T è entro i primi 3 anni, usiamo solo lambda_1 (lambda_3Y)
    if T <= n3Y:
        return np.exp(-lambda_1 * T) # Equivale a e^(-∫ from 0 to T of lambda_1 dt)
    # Altrimenti, usiamo lambda_1 per i primi 3 anni e lambda_2 (lambda_5Y) per i restanti anni fino a T
    else:
        return np.exp(-lambda_1 * n3Y) * np.exp(-lambda_2 * (T - n3Y)) # Equivale a e^(-∫ from 0 to 3 of lambda_1 dt) * e^(-∫ from 3 to T of lambda_2 dt)

In [89]:
# Calcoliamo le probabilità di sopravvivenza per ciascun anno
prob_survival_1y = calculate_survival_probability(lambda_3Y, lambda_5Y, 1, n3Y)
prob_survival_2y = calculate_survival_probability(lambda_3Y, lambda_5Y, 2, n3Y)
prob_survival_3y = calculate_survival_probability(lambda_3Y, lambda_5Y, 3, n3Y)
prob_survival_4y = calculate_survival_probability(lambda_3Y, lambda_5Y, 4, n3Y)
prob_survival_5y = calculate_survival_probability(lambda_3Y, lambda_5Y, 5, n3Y)

In [90]:
data = [['1y', prob_survival_1y, prob_survival_1y*100],
        ['2y', prob_survival_2y, prob_survival_2y*100],
        ['3y', prob_survival_3y, prob_survival_3y*100], 
        ['4y', prob_survival_4y, prob_survival_4y*100], 
        ['5y', prob_survival_5y, prob_survival_5y*100]] 
df = pd.DataFrame(data, columns=['Year', 'Prob. Survival', '% Prob. Survival'])
df = df.style.set_caption('Probability of survival')
df


,Year,Prob. Survival,% Prob. Survival
0,1y,0.958120,95.812004
1,2y,0.917994,91.799402
2,3y,0.879548,87.954846
3,4y,0.853262,85.326238
4,5y,0.827762,82.776189


---

# Parte II
<b>Sia X un sottostante con dinamica Bachelier in misura risk-neutra
dove σX è una costante positiva, e WX è un moto Browniano standard sotto Q. Si assuma il punto di vista di una banca che stipula un forward su X, ovvero un contratto secondo cui ad una data futura T la banca ricever`a XT pagando uno strike prefissato K. Si ipotizzi che la controparte sia il nome sottostante i CDS della Parte I, che abbia una frazione di loss given default attesa lgd, e che con essa non sia attivo nessun accordo di collateralizzazione.</b>

## Parte II.I
<b>Supponendo che la banca non abbia altri contratti in essere con la contropar- te, si scriva una formula per il Credit Valuation Adjustment unilaterale per unità di sottostante, esplicita a meno di un integrale, e la si implementi per approssimare numericamente il CVA della posizione al tempo 0, per i seguenti valori dei parametri (y indica l’unità di misura “anni”):

- lgd = 60%
- X0 = 2000 EUR
- σX = 300 EUR y−1/2
- K=1900EUR
- T=1y.</b>

In [91]:
# Parametri principali del problema
lgd = 0.60  # Loss given default
X_0 = 2000  # Valore iniziale del sottostante
k = 1900  # Strike price del forward
sigma = 300  # Volatilità annuale
T = 1  # Maturità in anni
r = 0.03  # Tasso risk-free annuale
lambda_3Y = 0.0428  # Intensità del processo di Poisson per il default
t = 0

La formula per il prezzo di un derivato secondo la dinamica di Bachelier è:

$ \text{Prezzo} = (S - K) \cdot \Phi(d) + \sigma \sqrt{T - t} \cdot \phi(d) $
<br>
<br>
   dove 
   - $ d = \frac{S - K}{\sigma \sqrt{T - t}} $
   - $S$ è il prezzo corrente del sottostante
   - $K$ è il prezzo di esercizio 
   - $\sigma$ è la volatilità, 
   - $T$ è il tempo alla scadenza
   - $t$ è il tempo corrente, 
   - $\Phi$ è la funzione di distribuzione cumulativa della normale standard 
   - $\phi$ è la funzione di densità di probabilità della normale standard.

In [92]:
def bachelier(X_0, k, sigma, T, tempo):
    d1 = (X_0 - k) / (sigma * np.sqrt(T - tempo))
    return (X_0 - k) * norm.cdf(d1) + sigma * np.sqrt(T - tempo) * norm.pdf(d1)

L'integranda usata per calcolare il CVA (closed formula) è:
<br>
<br>
$\int_t^T e^{-r s} \cdot \text{Prezzo}(s) \cdot \lambda \cdot e^{-\lambda s} \, ds$
<br>
<br>
dove 
- $r$ è il tasso risk-free
- $\lambda$ è l'intensità del processo di Poisson per il default
- $\text{Prezzo}(s)$ è il prezzo del derivato calcolato secondo la dinamica di Bachelier al tempo $s$

In [93]:
def integranda_cva(tempo, X_0, k, sigma, T, r, lambda_3Y):
    return np.exp(-r * tempo) * bachelier(X_0, k, sigma, T, tempo) * lambda_3Y * np.exp(-lambda_3Y * tempo)

Il CVA (closed formula) è dato da:
<br>
<br>
$\text{CVA} = -\text{LGD} \cdot \int_t^T e^{-r s} \cdot \text{Prezzo}(s) \cdot \lambda \cdot e^{-\lambda s} \, ds $

In [94]:
def cva(lgd, X_0, k, sigma, T, r, lambda_3Y, t):
    cva, _ = quad(integranda_cva, t, T, args=(X_0, k, sigma, T, r, lambda_3Y))
    return -lgd * cva

In [95]:
# CVA closed formula
cva_analitico = cva(lgd, X_0, k, sigma, T, r, lambda_3Y, t)
print(f"Valore analitico CVA: {cva_analitico:.4f} EUR")

Valore analitico CVA: -3.5128 EUR


## Parte II.II
<b>Per controllo, si calcoli un intervallo di confidenza al 98% per il CVA simulando 100,000 realizzazioni indipendenti del sottostante e dei tempi di fallimento.</b>

I tempi di default sono simulati secondo un processo esponenziale, che è l'inverso del processo di Poisson:
<br>
<br>
$\tau_i = -\frac{1}{\lambda} \ln(U_i)$
<br>
<br>
dove 
- $U_i$ è una variabile casuale uniformemente distribuita tra 0 e 1
- $\tau_i$ è il tempo di default simulato

In [96]:
# Simula i tempi di default secondo un processo di Poisson
def tempi_default(lambda_3Y, num_simulazioni):
    return np.random.exponential(1/lambda_3Y, num_simulazioni)

L'intervallo di confidenza per una media campionaria con un certo livello di confidenza è:
<br>
<br>
$\left[ \bar{X} - z_{\frac{\alpha}{2}} \cdot \frac{\sigma}{\sqrt{n}}, \bar{X} + z_{\frac{\alpha}{2}} \cdot \frac{\sigma}{\sqrt{n}} \right]$
<br>
<br>
dove 
- $\bar{X}$ è la media campionaria,
- $\sigma$ è lo scarto tipo campionario
- $n$ è la dimensione del campione
- $z_{\frac{\alpha}{2}}$ è il valore critico dalla distribuzione normale standard che corrisponde alla coda superiore di $ \frac{\alpha}{2}$

In [97]:
def intervallo_confidenza(valori_simulati, livello_confidenza):
    media = np.mean(valori_simulati)
    scarto_tipo = np.std(valori_simulati, ddof=1)
    z_score = norm.ppf(1 - livello_confidenza / 2)
    margine_errore = z_score * scarto_tipo / np.sqrt(len(valori_simulati))
    return media - margine_errore, media + margine_errore


In [98]:
# Simulazione Monte Carlo
num_simulazioni = 100000
tempi_default_simulati = tempi_default(lambda_3Y, num_simulazioni)
cva_simulati = [-lgd * np.exp(-r * tempo) * bachelier(X_0, k, sigma, T, tempo)
                if tempo < T else 0 for tempo in tempi_default_simulati]

In [99]:
# Calcolo dell'intervallo di confidenza al 98%
livello_confidenza = 0.98
intervallo_confidenza_cva = intervallo_confidenza(cva_simulati, livello_confidenza)

In [100]:
print(f"Valore medio CVA simulato: {np.mean(cva_simulati):.4f} EUR")
print(f"Intervallo di confidenza al {livello_confidenza*100:.0f}% per il CVA simulato: ({intervallo_confidenza_cva[0]:.4f}, {intervallo_confidenza_cva[1]:.4f}) EUR")

Valore medio CVA simulato: -3.5035 EUR
Intervallo di confidenza al 98% per il CVA simulato: (-3.5049, -3.5022) EUR
